<a href="https://colab.research.google.com/github/Aman78695/toxic-tweet-classification-NLP-Spacy-CountVectorizer-TfIdf-/blob/main/NLP_(toxic_tweets_classification)_using_spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
!pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [43]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import spacy
import string

In [44]:
df=pd.read_csv('/content/train.csv',engine='python', encoding='utf-8',error_bad_lines=False)

<ipython-input-44-b37ffa0ca891>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df=pd.read_csv('/content/train.csv',engine='python', encoding='utf-8',error_bad_lines=False)


In [45]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [46]:
df_new=df[['comment_text','toxic']].iloc[0:1000]

In [47]:
nlp=spacy.load('en_core_web_sm')

In [48]:
stp_words=nlp.Defaults.stop_words
print(stp_words)

{'first', 'becoming', 'give', 'might', 'move', '‘d', 'can', 're', 'again', 'whatever', 'anywhere', 'or', 'around', 'wherein', 'other', 'though', 'whom', 'still', 'above', 'go', 'although', 'whither', 'off', 'become', 'beyond', 'anyone', 'twelve', 'see', 'hence', 'yours', 'last', 'show', 'made', 'elsewhere', 'once', 'into', 'indeed', 'whoever', 'few', 'under', 'too', 'beside', 'everything', 'full', 'anyway', 'except', 'side', '’ll', 'how', 'also', 'perhaps', 'must', 'already', 'such', 'the', 'him', 'one', 'six', 'himself', 'hereupon', 'empty', 'should', 'name', 'latter', 'themselves', 'there', 'over', 'thereby', 'every', '‘re', 'do', 'until', 'thereupon', 'two', 'ca', 'at', 'why', 'well', 'afterwards', 'thus', 'seemed', 'together', 'hereby', '‘ll', 'became', '’re', 'their', 'his', 'ever', 'someone', 'many', 'really', 'been', 'others', 'which', 'sixty', '‘s', 'nine', 'ourselves', 'much', 'often', 'enough', '‘m', 'are', 'behind', 'get', 'but', 'with', 'almost', 'besides', "'d", 'have', 's

In [49]:
import nltk
import re
nltk.download('punkt')
punct=string.punctuation
print(punct)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [50]:
df_new['comment_text']=df_new['comment_text'].apply(lambda x: re.sub('[^a-zA-Z]',' ',x))

In [52]:
def spacy_tokenizer(sent):
  doc=nlp(sent)
  tokens=[word.lemma_.lower().strip() for word in doc]
  tokens=[word for word in tokens if word not in stp_words and word not in punct]
  return(tokens)

In [51]:
df_new['comment_text'][0]

'Explanation Why the edits made under my username Hardcore Metallica Fan were reverted  They weren t vandalisms  just closure on some GAs after I voted at New York Dolls FAC  And please don t remove the template from the talk page since I m retired now             '

In [ ]:
spacy_tokenizer('''Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27''')

In [53]:
count_vector=CountVectorizer(tokenizer=spacy_tokenizer)

In [54]:
from sklearn.model_selection import train_test_split
X=df_new['comment_text']
y=df_new['toxic']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [55]:
X_train_vectors=count_vector.fit_transform(X_train)
X_test_vectors=count_vector.transform(X_test)

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [66]:
count_vector.get_feature_names_out().shape

(6261,)

In [ ]:
count_vector.vocabulary_

In [58]:
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression()

In [76]:
X_train[15]

'   Juelz Santanas Age  In       Juelz Santana was    years old  then came February   th  which makes Juelz turn    making songs with The Diplomats  The third neff to be signed to Cam s label under Roc A Fella  In       he was    years old coming out with his own singles   Santana s Town   and   Down    So yes  he is born in       He really is  how could he be older then Lloyd Banks  And how could he be    when his birthday passed  The homie neff is    years old               Juelz death  god forbid if your thinking about that  equals     Go to your caculator and stop changing his year of birth  My god  '

In [67]:
X_train_vectors[0].shape

(1, 6261)

In [84]:
X_train_vectors.shape

(800, 6261)

In [68]:
clf.fit(X_train_vectors,y_train)

LogisticRegression()

In [82]:
pred=clf.predict(X_test_vectors)

In [83]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,pred)

0.915

In [80]:
df_new['comment_text'][884]

'    I TAKE OFFENSE    I take great offense that some pompous  sanctimonious twit thinks that no one should be allowed to express appreciation for women like the Gibson Girl as the   fine featured   and   achingly beautiful   creatures that they are     I don t know what kind of totalitarian feminazi dystopian agenda you are trying to foist on people  but don t do it here   This encyclopedia is based on free speech and a free exchange of information  not bogus political correctness and schoolmarmish thought policing   I ll take a Gibson Girl over you anyday  '

In [85]:
tfidf_vector=TfidfVectorizer(tokenizer=spacy_tokenizer)
x_train_vec=tfidf_vector.fit_transform(X_train)
x_test_vec=tfidf_vector.transform(X_test)

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
clf.fit(x_train_vec,y_train)

In [ ]:
predict=clf.predict(x_test_vec)

In [ ]:
accuracy_score(y_test,x_test_vec)